In [1]:
import praw
import pandas as pd
import requests
from praw.models import MoreComments
import datetime as dt
from psaw import PushshiftAPI
from tqdm import tqdm
import numpy as np
import pickle
from utils import crawler_utils
from utils.crawler_utils import time_initialization, club_abbr, club_reddit_abbr, transform_df, parse_dates

In [2]:
reddit = praw.Reddit(client_id='bCnE1U61Wqixgs2wy28POg', client_secret='vEY7k3_j7o3PZZvP-tEt6DnhWr1x5A', user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36')

In [7]:
filename = f'../data/MIL.csv'    
data = pd.read_csv(filename)
data.tail()

,author,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url,date
8117,Claija79,1.627752e+09,ovb8tj,5.0,1.0,ACMilan,[Longo] Gasperini definitely open to the sale ...,1.0,https://twitter.com/86_longo/status/1421518730...,2021-08-01 01:27:48
8118,Hass_s,1.627752e+09,ovb2jn,11.0,1.0,ACMilan,"[ Antonio Vitiello ] Gasperini: "" Ilicic ? at ...",1.0,https://twitter.com/AntoVitiello/status/142151...,2021-08-01 01:17:23
8119,Hass_s,1.627751e+09,ovapqy,20.0,1.0,ACMilan,[Pietro Mazzara] Rafael Leao is not on the mar...,1.0,https://twitter.com/pietromazzara/status/14215...,2021-08-01 00:57:26
8120,Hass_s,1.627749e+09,ova4u2,11.0,1.0,ACMilan,Comunicato Ufficiale: Lorenzo Colombo | AC Mil...,1.0,https://www.acmilan.com/it/news/articoli/media...,2021-08-01 00:24:21
8121,NotCoolNickname,1.627748e+09,ov9uea,0.0,1.0,ACMilan,Maldini negotiations leaked on YouTube,1.0,https://youtu.be/Ds-7z7tPBCg,2021-08-01 00:07:57


In [6]:
for club_name in tqdm(club_abbr):
    filename = f'./data/{club_name}.csv'    
    data = pd.read_csv(filename)
    data_comments_50 = data.loc[data['num_comments']>=50]
    print(club_name)
    print(data_comments_50.shape[0])

 38%|███▊      | 6/16 [00:00<00:00, 27.65it/s]

MIL
681
AMD
144
BAR
1231
DOR
316
CHE
3950
MUN
523


 50%|█████     | 8/16 [00:00<00:00, 20.80it/s]

INT
192
ARS
8159
JUV
446


 81%|████████▏ | 13/16 [00:00<00:00, 21.72it/s]

LIV
4393
MNC
956
PSG
276
RMD
1417


100%|██████████| 16/16 [00:00<00:00, 22.44it/s]

MNU
5271
ROM
2
TOT
9


In [3]:
auth = requests.auth.HTTPBasicAuth('bCnE1U61Wqixgs2wy28POg','vEY7k3_j7o3PZZvP-tEt6DnhWr1x5A')

data = {'grant_type': 'password',
        'username': 'Delta_Wang11',
        'password': 'delta113420'
        }

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                     auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization':f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me',headers=headers)

<Response [200]>

In [4]:
res = requests.get("https://oauth.reddit.com/r/soccer/search?q=flair_name%3A%22Post%20Match%20Thread%22&restrict_sr=1000&sort=new&type=link",headers=headers)

In [5]:
res.json()['data']['children']

[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'soccer',
   'selftext': "#**FT: Libya  [0-3](#bar-3-white)  Egypt**\n\n*Egypt goalscorers: Ahmed Abou El Fotouh (40'), Mostafa Mohamed (45+4'), Ramadan Sobhi (72')*\n\n\n\n--------\n\n**Venue:** 28 March Stadium\n\n[Auto-refreshing reddit comments link](http://www.reddit-stream.com/comments/q62rk0)\n\n---------\n\n[](#icon-notes-big) **LINE-UPS**\n\n**Libya**\n\nMohamed Nashnoush, Ali Salama, Motasem Sabbou, Sanad El Werfelli, Abdalla Sherif, Mohamed Hamdoud, Faisal Al Badri, Elmutasem El Masrati, Mohammed Al Tabal, Mohamed Zubya, Muaid Ellafi.\n\n**Subs:** Senosi Alhadi, Mohammed Ali Soulah, Mohamed Al Tarhuni, Ali Youssef, Murad Al Woheshi, Tahir Bin Amir, Mohammed Touhami, Omar Al Khouja, Mohamed El Monir, Noor-Eildein Ahmed.\n\n^____________________________\n\n**Egypt**\n\nMohamed El-Shenawy, Ahmed Hegazi, Akram Tawfik, Ahmed Abou El Fotouh, Mahmoud Hamdy, Abdallah Said, Mohamed Elneny, Omar Marmoush, Amro Elsouli

In [7]:
df = pd.DataFrame()
for post in res.json()['data']['children']:
    # append relevant data to dataframe
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'id': post['data']['id'],
        'url':post['data']['url'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'num_comments':post['data']['num_comments'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score'],
        'created': post['data']['created']
    }, ignore_index=True)

In [9]:
df

,created,downs,id,num_comments,score,selftext,subreddit,title,ups,upvote_ratio,url
0,1.633986e+09,0.0,q65low,32.0,36.0,#**FT: Libya [0-3](#bar-3-white) Egypt**\n\n...,soccer,Post-Match Thread: Libya 0-3 Egypt | World Cup...,36.0,0.90,https://www.reddit.com/r/soccer/comments/q65lo...
1,1.633985e+09,0.0,q65bnj,107.0,83.0,#**FT: Latvia [1-2](#bar-3-white) Turkey**\n...,soccer,Post-Match Thread: Latvia 1-2 Turkey | World C...,83.0,0.93,https://www.reddit.com/r/soccer/comments/q65bn...
2,1.633985e+09,0.0,q65b2x,19.0,30.0,#**FT: Slovenia [1-2](#bar-3-white) Russia**...,soccer,Post-Match Thread: Slovenia 1-2 Russia | World...,30.0,0.82,https://www.reddit.com/r/soccer/comments/q65b2...
3,1.633985e+09,0.0,q659n7,49.0,68.0,#**FT: Norway [2-0](#bar-3-white) Montenegro...,soccer,Post-Match Thread: Norway 2-0 Montenegro | Wor...,68.0,0.95,https://www.reddit.com/r/soccer/comments/q659n...
4,1.633985e+09,0.0,q65948,1.0,17.0,**Belarus** 0-2 **Czech Republic**\n\n*Czech R...,soccer,Post-Match Thread: Belarus 0-2 Czech Republic ...,17.0,0.83,https://www.reddit.com/r/soccer/comments/q6594...
5,1.633985e+09,0.0,q658fu,44.0,118.0,#**FT: North Macedonia [0-4](#bar-3-white) G...,soccer,Post-Match Thread: North Macedonia 0-4 Germany...,118.0,0.97,https://www.reddit.com/r/soccer/comments/q658f...
6,1.633985e+09,0.0,q657uv,38.0,60.0,#**FT: Netherlands [6-0](#bar-3-white) Gibra...,soccer,Post-Match Thread: Netherlands 6-0 Gibraltar |...,60.0,0.95,https://www.reddit.com/r/soccer/comments/q657u...
7,1.633985e+09,0.0,q657cx,13.0,24.0,#**FT: Estonia [0-1](#bar-3-white) Wales**\n...,soccer,Post-Match Thread: Estonia 0-1 Wales | World C...,24.0,0.83,https://www.reddit.com/r/soccer/comments/q657c...
8,1.633985e+09,0.0,q6576u,134.0,55.0,# [](#sprite6-p43) **Croatia** [2 - 2](#bar-3-...,soccer,Post-Match Thread: Croatia 2-2 Slovakia | FIFA...,55.0,0.89,https://www.reddit.com/r/soccer/comments/q6576...
9,1.633985e+09,0.0,q656k4,25.0,48.0,#**FT: Romania [1-0](#bar-3-white) Armenia**...,soccer,Post-Match Thread: Romania 1-0 Armenia | World...,48.0,0.91,https://www.reddit.com/r/soccer/comments/q656k...


In [10]:
comments = []
for uid in df.id:
    submission = reddit.submission(id=uid)
    for top_level_comment in tqdm(submission.comments):
        if isinstance(top_level_comment, MoreComments):
            for cid in top_level_comment.children:
                comments.append(reddit.comment(cid).body)
        else:
            comments.append(top_level_comment.body)

 99%|█████████▉| 155/156 [00:36<00:00,  4.27it/s]


KeyboardInterrupt: 

In [10]:
len(comments)

1581

In [2]:
def time_initialization():
    start_time_list = []
    end_time_list = []
    year = [2020, 2020, 2020, 2020, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021]
    month = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8]
    end_day = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

    for i in range(12):
        start_time_list.append(int(dt.datetime(year[i], month[i], 1).timestamp()))
        end_time_list.append(int(dt.datetime(year[i], month[i], end_day[i]).timestamp()))

    return start_time_list, end_time_list

In [3]:
api = PushshiftAPI()

club_reddit_abbr = ['ACMilan', 'atletico', 'Barca', 'borussiadortmund', 'chelseafc', 'fcbayern', 'FCInterMilan', 'Gunners',
             'Juve', 'LiverpoolFC', 'MCFC', 'psg', 'realmadrid', 'reddevils', 'roma', 'Tottenham']

club_abbr = ['MIL', 'AMD', 'BAR', 'DOR', 'CHE', 'MUN', 'INT', 'ARS',
             'JUV', 'LIV', 'MNC', 'PSG', 'RMD', 'MNU', 'ROM', 'TOT']

MIL_list = []
AMD_list = []
BAR_list = []
DOR_list = []
CHE_list = []
MUN_list = []
INT_list = []
ARS_list = []
JUV_list = []
LIV_list = []
MNC_list = []
PSG_list = []
RMD_list = []
MNU_list = []
ROM_list = []
TOT_list = []
all_list = [MIL_list, AMD_list, BAR_list, DOR_list, CHE_list, MUN_list, INT_list, ARS_list,
            JUV_list, LIV_list, MNC_list, PSG_list, RMD_list, MNU_list, ROM_list, TOT_list]


In [37]:
start_time, end_time = time_initialization()

In [40]:
for club_index in tqdm(range(len(all_list))):
    temp_list = []
    for i in tqdm(range(len(start_time))):
        temp_array = list(api.search_submissions(after=start_time[i], before=end_time[i],
                                                                subreddit=club_reddit_abbr[club_index],
                                                                filter=['url', 'author', 'title', 'subreddit', 'id',
                                                                        'num_comments', 'score', 'ups', 'upvote_ratio'])
                                         )
        temp_aa = np.array(temp_array)
        temp_list.append(temp_aa)
    
    file = open(f'./data/{club_abbr[club_index]}.pickle', 'wb')
    pickle.dump(temp_list, file)
    file.close()

  6%|▋         | 1/16 [01:21<20:16, 81.11s/it]


KeyboardInterrupt: 

In [45]:
type(temp_list[0][0][0])

str

In [21]:
type(temp_array[0])

list

In [10]:
type(temp_list[0][0])

psaw.PushshiftAPI.submission

In [76]:
a_list = np.array(all_list, dtype=object)

In [77]:
type(a_list[0])

list

In [45]:
file = open(f'./data/{club_abbr[club_index]}.pickle', 'wb')
pickle.dump(temp_list[0][0], file)
file.close()

PicklingError: Can't pickle <class 'psaw.PushshiftAPI.submission'>: attribute lookup submission on psaw.PushshiftAPI failed

In [54]:
Barca = np.array(Barca_list, dtype=object)

In [19]:
club_abbr = ['BARCELONA']

In [31]:
import pickle
file = open(f'./data/MIL.pickle', 'wb')
pickle.dump(Barca, file)
file.close()

In [55]:
type(Barca[0][0])

psaw.PushshiftAPI.submission

In [22]:
a=np.load('BARCELONA.npy',allow_pickle=True)
a=a.tolist()

In [46]:
with open('./data/MIL.pickle', 'rb') as file:
    a = pickle.load(file)
print(a)

[array([['Samkazi23', 1601394725, 'j21vyv', ...,
        'https://twitter.com/ACMReports/status/1310969921397051396?s=09',
        1601365925.0,
        {'author': 'Samkazi23', 'created_utc': 1601394725, 'id': 'j21vyv', 'num_comments': 9, 'score': 1, 'subreddit': 'ACMilan', 'title': 'Jens Petter Hauge has just arrived to the hotel after landing in Malpensa. He rests now, and tomorrow he will have the medical examinations before signing his contract with AC Milan.', 'upvote_ratio': 1.0, 'url': 'https://twitter.com/ACMReports/status/1310969921397051396?s=09', 'created': 1601365925.0}],
       ['Gaverini', 1601394196, 'j21pjj', ...,
        'https://i.redd.it/p5ud8vafw3q51.png', 1601365396.0,
        {'author': 'Gaverini', 'created_utc': 1601394196, 'id': 'j21pjj', 'num_comments': 7, 'score': 1, 'subreddit': 'ACMilan', 'title': 'Favourite AC Milan Goal: Final Results', 'upvote_ratio': 1.0, 'url': 'https://i.redd.it/p5ud8vafw3q51.png', 'created': 1601365396.0}],
       ['Dexonic', 16013907

In [53]:
a[0][0]

array(['Samkazi23', 1601394725, 'j21vyv', 9, 1, 'ACMilan',
       'Jens Petter Hauge has just arrived to the hotel after landing in Malpensa. He rests now, and tomorrow he will have the medical examinations before signing his contract with AC Milan.',
       1.0,
       'https://twitter.com/ACMReports/status/1310969921397051396?s=09',
       1601365925.0,
       {'author': 'Samkazi23', 'created_utc': 1601394725, 'id': 'j21vyv', 'num_comments': 9, 'score': 1, 'subreddit': 'ACMilan', 'title': 'Jens Petter Hauge has just arrived to the hotel after landing in Malpensa. He rests now, and tomorrow he will have the medical examinations before signing his contract with AC Milan.', 'upvote_ratio': 1.0, 'url': 'https://twitter.com/ACMReports/status/1310969921397051396?s=09', 'created': 1601365925.0}],
      dtype=object)

In [61]:
a[0][8][10]['id']

'j1y1f6'

In [52]:
len(a[4])

948

In [ ]:
api = PushshiftAPI()

start_time = int(dt.datetime(2021, 1, 1).timestamp())
end_time = int(dt.datetime(2021, 1, 2).timestamp())

b_list =[]
b_list.append(list(api.search_submissions(after=start_time, before=end_time, subreddit='Barca', 
filter=['url','author', 'title', 'subreddit'])))

start_time = int(dt.datetime(2021, 1, 2).timestamp())
end_time = int(dt.datetime(2021, 1, 3).timestamp())
b_list.append(list(api.search_submissions(after=start_time, before=end_time, subreddit='Barca', 
filter=['url','author', 'title', 'subreddit'])))

start_time = int(dt.datetime(2021, 1, 3).timestamp())
end_time = int(dt.datetime(2021, 1, 4).timestamp())
b_list.append(list(api.search_submissions(after=start_time, before=end_time, subreddit='Barca', 
filter=['url','author', 'title', 'subreddit'])))

In [23]:
temp_b = np.array(b_list, dtype=object)

In [26]:
type(temp_b[0][0])

psaw.PushshiftAPI.submission

In [58]:
b_list

[[submission(author='james_downpick_hetfi', created_utc=1609513874, subreddit='Barca', title='[DISCUSSION] I saw a post asking if Ronaldo is a barça legend, we all know Ronaldinho is indeed a barça legend, but the questions is, Is Ronaldinho in the top 5 of biggest Barça legends', url='https://www.reddit.com/r/Barca/comments/kocr0w/discussion_i_saw_a_post_asking_if_ronaldo_is_a/', created=1609485074.0, d_={'author': 'james_downpick_hetfi', 'created_utc': 1609513874, 'subreddit': 'Barca', 'title': '[DISCUSSION] I saw a post asking if Ronaldo is a barça legend, we all know Ronaldinho is indeed a barça legend, but the questions is, Is Ronaldinho in the top 5 of biggest Barça legends', 'url': 'https://www.reddit.com/r/Barca/comments/kocr0w/discussion_i_saw_a_post_asking_if_ronaldo_is_a/', 'created': 1609485074.0}),
  submission(author='damidiola', created_utc=1609508921, subreddit='Barca', title='Preferred signing', url='https://www.reddit.com/r/Barca/comments/kobjdd/preferred_signing/', c

In [12]:
type(b_list[0][0])

psaw.PushshiftAPI.submission

In [13]:
B_array = np.array(b_list, dtype=object)

In [15]:
type(B_array[0][0][0])

str

In [66]:
B_array[0][0]

array(['james_downpick_hetfi', 1609513874, 'Barca',
       '[DISCUSSION] I saw a post asking if Ronaldo is a barça legend, we all know Ronaldinho is indeed a barça legend, but the questions is, Is Ronaldinho in the top 5 of biggest Barça legends',
       'https://www.reddit.com/r/Barca/comments/kocr0w/discussion_i_saw_a_post_asking_if_ronaldo_is_a/',
       1609485074.0,
       {'author': 'james_downpick_hetfi', 'created_utc': 1609513874, 'subreddit': 'Barca', 'title': '[DISCUSSION] I saw a post asking if Ronaldo is a barça legend, we all know Ronaldinho is indeed a barça legend, but the questions is, Is Ronaldinho in the top 5 of biggest Barça legends', 'url': 'https://www.reddit.com/r/Barca/comments/kocr0w/discussion_i_saw_a_post_asking_if_ronaldo_is_a/', 'created': 1609485074.0}],
      dtype=object)

In [6]:
a = [['a'],['b'],['c']]
a1 = np.array(a[0],dtype=object)
type(a1[0])

str

In [ ]:
with open('./data/MIL.pickle', 'rb')

In [3]:
%%time
all_comments = []
uid = 'j0uxd6'
submission = reddit.submission(id=uid)
submission.comments.replace_more(limit=None)
for comment in submission.comments.list():
    all_comments.append(comment.body)

CPU times: user 184 ms, sys: 24.2 ms, total: 208 ms
Wall time: 10 s


In [4]:
len(all_comments)

531